In [1]:
import arcpy
from arcpy import env
import pandas as pd
import numpy as np

In [2]:
env_path = r"D:\ArcgisData\pred_soildeep\pre_database\pred_soildeep_gz.gdb"
env.workspace = env_path
arcpy.ListRasters()

['AP', 'NDVI', 'NIGHT', 'PRE', 'SOILQS', 'TMP']

In [3]:
# 构造经纬度信息
desc = arcpy.Describe("NDVI")
origin_x = desc.extent.XMin
origin_y = desc.extent.YMax
pixel_width = desc.meanCellWidth
pixel_height = desc.meanCellHeight
print(origin_x,origin_y,pixel_width,pixel_height)

58416.702553160896 3220372.9676713645 30.0 30.0


In [4]:
stander_array = arcpy.RasterToNumPyArray("NDVI")
stander_array.shape

(16001, 19678)

In [5]:
# 经度
array_x = np.zeros(stander_array.shape, dtype=np.float64)
array_x[:, 0] = origin_x+pixel_width/2
for i in range(1, stander_array.shape[1]):
    array_x[:, i] = array_x[:, i-1] + pixel_width
print(array_x.shape,array_x[0,1],array_x[0,-1])

(16001, 19678) 58461.702553160896 648741.702553161


In [6]:
# 纬度
array_y = np.zeros(stander_array.shape,dtype=np.float64)
array_y[0] = origin_y+pixel_height/2
for i in range(1, stander_array.shape[0]):
    array_y[i] = array_y[i-1] - pixel_height
print(array_y.shape,array_y[0][0],array_y[-1][0])

(16001, 19678) 3220387.9676713645 2740387.9676713645


In [45]:
def split_array_into_eight(arr):
    """
    将 NumPy 数组分为八份。

    参数:
        arr (numpy.ndarray): 要分割的数组

    返回:
        list: 包含八个 NumPy 数组的列表
    """
    
    if len(arr.shape) != 2:
        raise ValueError("Input array must be 2-dimensional")

    # 计算每部分的行数
    rows_per_part = arr.shape[0] // 2
    parts = []

    for i in range(2):
        start_index = i * rows_per_part

        # 对于最后一个部分，取所有剩余的行
        if i == 2:
            parts.append(arr[start_index:])
        else:
            end_index = start_index + rows_per_part
            parts.append(arr[start_index:end_index])
    
    return parts
# 将预测结果输出为栅格数据
def put_raster(in_array):
    out_raster = arcpy.NumPyArrayToRaster(
        in_array,
        arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMin),
        30,
        30,
    )
    return out_raster
# 将预测结果输出为栅格数据,有分割处理
def put_raster_cut2(in_array):
    out_raster = arcpy.NumPyArrayToRaster(
        in_array,
        arcpy.Point(arcpy.env.extent.XMin, arcpy.env.extent.YMax-(in_array.shape[0]*30)),
        30,
        30,
    )
    return out_raster


In [46]:
# 确定范围
env.extent = 'NDVI'


In [47]:
x_split = split_array_into_eight(array_x)
y_split = split_array_into_eight(array_y)

In [49]:

put_raster(y_split[1]).save(f"LON_2")


In [ ]:
import arcpy
from arcpy.sa import *

# 设置工作空间和覆盖输出

# 指定输入栅格的路径
raster_path = "LON_9"

# 创建一个 FlowDirection 栅格。我们只是使用这个工具来获取像元的 x 坐标，实际上我们并不关心流向。
flow_direction_raster = FlowDirection(raster_path)

# 使用 Lookup 工具获取每个像元的 x 坐标
longitude_raster = Lookup(flow_direction_raster, "X")

# 保存输出栅格
longitude_raster.save("path_to_save_longitude_raster.tif")
